- **Generic function**
  - A central concept in object abstraction is a generic function, which is a function that can accept values of multiple different types. We will consider threee different techniques for implementing generic functions: shared interfaces, type dispatching, and type coercion. In the processing of building up these concepts, we will also discover features of the Python object system that support the creation of generic functions. 

### 2.7.1 String Conversion 

The str constructor often coincides with repr, but provides a more interpretable text representation in some cases. For instance, we see a difference between str and repr with dates. 

In [1]:
from datetime import date
tues = date(2011, 9, 12)
repr(tues)

'datetime.date(2011, 9, 12)'

In [2]:
str(tues)

'2011-09-12'

### 2.7.2 Special Methods

- **Sequence operations**
  - We have seen that we can call tghe len function to determine the length of a sequence. 

In [3]:
len('Go Bears!')

9

In [4]:
'Go Bears!'.__len__()

9

- The len function invokes the `__len__` method of its argument to determine its length. All built-in sequence types implement this method. 

- Python uses a sequence's length to determine its truth value, if it does not provide a `__bool__` method. Empty sequences are false, while non-empty sequences are true. 

In [5]:
bool('')

False

In [6]:
bool([])

False

In [9]:
bool(([],))

True

In [8]:
bool([()])

True

In [10]:
bool('Go Bears!')

True

- **`__getitem__`**
  - The `__getitem__` method is invoked by the element selection operator, but it can also be invoked directly. 

In [12]:
'Go Bears!'[3]

'B'

In [13]:
'Go Bears!'.__getitem__(3)

'B'

- **Callable objects**
  - In Python, functions are first-class objects, so they can be passed around as data and have attributes like any other object. 
  - Python also us to define objects that can be "called" like functions by including a `__call__` method. With this method, we can define a class that behaves like a higher-order function.

In [14]:
def make_adder(n):
    def adder(k):
        return n + k
    return adder

In [15]:
add_three = make_adder(3)

In [16]:
add_three(4)

7

- We can create an adder class that defines a `__call__` method to provide the same functionality. 

In [18]:
class Adder(object):
    def __init__(self, n):
        self.n = n
    def __call__(self, k):
        return self.n + k

add_three_obj = Adder(3)
add_three_obj(4)

7

### 2.7.3 Multiple Representations

- Abstraction barries allow us to separate the use and the representation of data. However, in large programs, it may not always make sense to speak of "the underlying representation" for a daya type in a program.
  - For one thing, there might be more than one useful representation ofr a data object.
  - we might like to design systems that can deal with multiple representations.

In [19]:
class Number:
    def __add__(self, other):
        return self.add(other)
    def __mul__(self, other):
        return self.mul(other)

- This class requires that Number objects have add and mul methods, but does not define them. Moreover, it does not have an `__init__` method. 
- **The purpose** of Number is not to be instantiated directly, but instead to serve as a superclass of varous specific number classes. Our next task is to define add and mul appropriately for complex numbers. 

- The Complex class inherits from Number and describes arithmetic for complex numbers.

In [20]:
class Complex(Number):
    def add(self, other):
        return ComplexRI(self.real + other.real, self.imag + other.imag)
    def mul(self, other):
        magnitude = self.magnitude * other.magnitude
        return ComplexMA(magnitude, self.angle + other.angle)

- **Interfaces**
  - Object attributes, which are a form of message passing, allows different data types to respond to the same message in different ways. A shared set of messages that elicit similar behavior from different classes is a powerful method of abstraction. An interface is a set of shared attribute names, along with a specification of their behavior. 

- **Properties**
  - The requirement that two or more attribute values maintain a fixed relationship with each other is a new problem. 
  - The @property decorator allows functions to be called without call expression syntax.

In [21]:
from math import atan2
class ComplexRI(Complex):
    def __init__(self, real, imag):
        self.real = real
        self.imag = imag
    @property
    def magnitude(self):
        return (self.real ** 2 + self.imag ** 2) ** 0.5
    @property
    def angle(self):
        return atan2(self.imag, self.real)
    def __repr__(self):
        return 'ComplexRI({0:g}, {1:g})'.format(self.real, self.imag)

In [22]:
ri = ComplexRI(5, 12)

In [23]:
ri.real

5

In [24]:
ri.magnitude

13.0

In [27]:
from math import sin, cos, pi
class ComplexMA(Complex):
    def __init__(self, magnitude, angle):
        self.magnitude = magnitude
        self.angle = angle
    @property
    def real(self):
        return self.magnitude * cos(self.angle)
    @property
    def imag(self):
        return self.magnitude * sin(self.angle)
    def __repr__(self):
        return 'ComplexMA({0:g}, {1:g} * pi)'.format(self.magnitude, self.angle/pi)

In [28]:
ComplexRI(1, 2) + ComplexMA(2, pi/2)

ComplexRI(1, 4)

### 2.7.4 Genecric Functions
- Generic functions are methods or functions that apply to arguments of different types. 

In [29]:
from fractions import gcd
class Rational(Number):
    def __init__(self, numer, denom):
        g = gcd(numer, denom)
        self.numer = numer // g
        self.denom = denom // g
    def __repr__(self):
        return 'Rational({0}, {1})'.format(self.numer, self. denom)
    def add(self, other):
        nx, dx = self.numer, self.denom
        ny, dy = other.numer, other.denom
        return Rational(nx * dy + ny * dx, dx * dy)
    def mul(self, other):
        numer = self.numer * other.numer
        denom = self.denom * other.denom
        return Rational(numer, denom)

- **Type dispatching**
  - The idea of type dispatching is to write functions that inspext the type of arguments they receive, then execute code that is appropriate for those types. 

In [30]:
def is_real(c):
    """Return whether c is a real number with no imginary part."""
    if isinstance(c, ComplexRI):
        return c.imag == 0
    elif isinstance(c, ComplexMA):
        return c.angle%pi == 0

- The role of type dispatching is to ensure that these cross-type operations are used at appropriate times. Below, we rewrite the Number superclass to use type dispatching its `__add__` and `__mul__` methods.

In [31]:
Rational.type_tag = 'rat'
Complex.type_tag = 'com'